***Train the Model***

In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler 
from sklearn.externals import joblib
from sklearn.metrics import mean_absolute_error

# Load our data set 
df = pd.read_csv("house_data1.csv")

#Create the X and y arrays
X = df[["sq_feet", "num_bedrooms", "num_bathrooms"]]
y = df[["sale_price"]]

Using TensorFlow backend.


In [2]:
# Create a scaler for the inputs and outputs
X_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the input and output data
X[X.columns] = X_scaler.fit_transform(X[X.columns])
y[y.columns] = y_scaler.fit_transform(y[y.columns])

# Split the data set in a training set (75%) and a test set (25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Create a Neural Network model
model = Sequential() # Most common Keras API, define one layer at a time. 
model.add(Dense(50, input_dim=3, activation="relu"))
model.add(Dense(100, activation="relu"))
model.add(Dense(50, activation="relu"))
model.add(Dense(1, activation="linear"))
model.compile(
    loss = "mean_squared_error",
    optimizer="SGD"
)

# Train the model 
model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=8,
    shuffle=True,
    verbose=2
)

# Save the scalers to files so we can use it to pre-process new data later
joblib.dump(X_scaler, "X_scaler.pkl")
joblib.dump(y_scaler, "y_scaler.pkl")

# Save the trained model to a file so we can use it to make predictions later 
model.save("house_value_model.h5")

# Report how well the model is performing
print("Model training results:")

# Report mean absolute error on the training set in a value scaled back to dollars so it's easier to understand
predictions_train = model.predict(X_train, verbose=0)

mse_train = mean_absolute_error(
    y_scaler.inverse_transform(predictions_train),
    y_scaler.inverse_transform(y_train)
)

print(f" - Training Set Error: {mse_train}")

# Report mean absolute error on the test set in a value scaled back to dollars so it's easier to understand
predictions_test = model.predict(X_test, verbose=0)

mse_test = mean_absolute_error(
    y_scaler.inverse_transform(predictions_test),
    y_scaler.inverse_transform(y_test)
)

print(f" - Test Set Error: {mse_test}")


C:\Users\maikh\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\maikh\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


C:\Users\maikh\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Epoch 1/50
 - 1s - loss: 0.0112
Epoch 2/50
 - 1s - loss: 5.2387e-04
Epoch 3/50
 - 1s - loss: 4.0357e-04
Epoch 4/50
 - 1s - loss: 3.7786e-04
Epoch 5/50
 - 1s - loss: 3.6541e-04
Epoch 6/50
 - 1s - loss: 3.5817e-04
Epoch 7/50
 - 1s - loss: 3.5249e-04
Epoch 8/50
 - 1s - loss: 3.4783e-04
Epoch 9/50
 - 1s - loss: 3.4648e-04
Epoch 10/50
 - 1s - loss: 3.4271e-04
Epoch 11/50
 - 1s - loss: 3.4277e-04
Epoch 12/50
 - 1s - loss: 3.4033e-04
Epoch 13/50
 - 1s - loss: 3.3863e-04
Epoch 14/50
 - 1s - loss: 3.3775e-04
Epoch 15/50
 - 1s - loss: 3.3752e-04
Epoch 16/50
 - 1s - loss: 3.3495e-04
Epoch 17/50
 - 1s - loss: 3.3523e-04
Epoch 18/50
 - 1s - loss: 3.3410e-04
Epoch 19/50
 - 1s - loss: 3.3253e-04
Epoch 20/50
 - 1s - loss: 3.3240e-04
Epoch 21/50
 - 1s - loss: 3.3139e-04
Epoch 22/50
 - 1s - loss: 3.3232e-04
Epoch 23/50
 - 1s - loss: 3.3131e-04
Epoch 24/50
 - 1s - loss: 3.2952e-04
Epoch 25/50
 - 1s - loss: 3.2929e-04
Epoch 26/50
 - 1s - loss: 3.2924e-04
Epoch 27/50
 - 1s - loss: 3.2949e-04
Epoch 28/50
 -

***Use the model***

In [5]:
from keras.models import load_model
from sklearn.externals import joblib

# Load our trained model
model1 = load_model("house_value_model.h5")

# Load the data scalers so that we can transform new data and prediction the same way as training data
X_scaler = joblib.load("X_scaler.pkl")
y_scaler = joblib.load("y_scaler.pkl")

# Define the house that we want to value (with the values in the same order as in the training data).
house_1 = [
    2000, # Size in Square Feet
    3, # Number of Bedrooms 
    2, # Number of Bathrooms
]

# Keras assumes we want to predict the values for multiple of houses at once, so it expects an array.
# We only want to value a single house, so it will be the only item in our array.
homes = [
    house_1
]

# Scale the new data like the training data
scaled_home_data = X_scaler.transform(homes)

# Make a prediction for each house in the homes array (but we only have one)
home_values = model1.predict(scaled_home_data)

# The prediction from the neural network will be scaled 0 to 1 just like the training data
# We need to unscale it using the same factor as we used to scale the training data
unscaled_home_values = y_scaler.inverse_transform(home_values)

# Since we are only predicting the price of one house, grab the first prediction returned
predicted_value = unscaled_home_values[0][0]

# Print the results
print("House details:")
print(f"- {house_1[0]} sq feet")
print(f"- {house_1[1]} bedrooms")
print(f"- {house_1[2]} bathrooms")
print(f"Estimated value: ${predicted_value:,.2f}")

House details:
- 2000 sq feet
- 3 bedrooms
- 2 bathrooms
Estimated value: $383,302.97
